# Views
- Vistas basadas en funciones
- Vistas basadas en clases

## Para crear una aplicacion en Django
- python manage.py startapp
- python manage.py startapp ecommerce

## Para entrar en el container
- docker exec -it hellodjango-web-1 bash
- muestra como entro: python@6533204c6cff:/app/src$ 

## CRUD
Create, Retrieve, Update, Delete

## Views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpRequest, HttpResponse

# Create your views here.
def home(request: HttpRequest) -> HttpResponse:
    return HttpResponse("Welcome to the E-commerce Home Page")

## Urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.home, name="home"),
]


## Config/urls.py

In [ ]:
urlpatterns = [
    path("up/", include("up.urls")),
    path("", include("pages.urls")),
    path("ecommerce/", include("ecommerce.urls")), #<-- Added line to include ecommerce URLs
    path("admin/", admin.site.urls),
]

## ecommerce/urls.py
Redirect

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.home, name="home"),
    path("redirect/", views.redirect_to_home, name="redirect_to_home"),
]


### Models.py

In [ ]:
from django.db import models

# Create your models here.
class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()

## Correr migraciones
- python manage.py makemigrations
- python manage.py migrate

In [ ]:
python@6533204c6cff:/app/src$ python manage.py makemigrations
Migrations for 'ecommerce':
  ecommerce/migrations/0001_initial.py
    + Create model ProductModel

- Migrate

In [ ]:
Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_length... OK
  Applying ecommerce.0001_initial... OK
  Applying sessions.0001_initial... OK

## Registrar modelos en Admin

In [ ]:
from django.contrib import admin

# Register your models here.
from .models import ProductModel

admin.site.register(ProductModel)

## Create user type Admin

In [ ]:
python manage.py createsuperuser

- Password must be at least 8 charachters
 It must contain at least 8 characters.

 - Endpoint admin
 http://0.0.0.0:8000/admin/login/?next=/admin/


## Tipos basicos de Vistas
Dentro de Users en admin
- Listas View donde ves los usuarios creados.
- Create View donse puedes insertar los datos para crear un usuario (add user)
- Retrive and Update View al dar click en un usuario creado, para modificarlo
- Delete View para elmininar

## Templates
Uso de templates en Views

In [ ]:
def product_model_list_view(request: HttpRequest) -> HttpResponse:
    products = ProductModel.objects.all()
    template = "ecommerce/product_list.html"
    context = {"products": products}
    return render(request, template, context)

Create the template:
 - src/pages/templates/ecommerce/product_list.html

In [ ]:
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Lista de Productos</title>
  </head>
  <body>
    {% for product in products %}
    <ul>
      <li>{{ product.title }} - ${{ product.price }}</li>
    </ul>
    {% endfor %}
  </body>
</html>


## Protegiendo los endpoints

- Add on config/settings.py the url as
- LOGIN_REDIRECT_URL = "/admin/login/"

and on views add the decorator as:

In [ ]:
from django.contrib.auth.decorators import login_required

# Create your views here.
@login_required(login_url='/admin/login/')
def product_model_list_view(request: HttpRequest) -> HttpResponse:
    products = ProductModel.objects.all()
    template = "ecommerce/product_list.html"
    context = {"products": products}
    
    if request.user.is_authenticated:
        template = "ecommerce/product_list.html"
    else:
        template = "ecommerce/product_list_public.html"
    
    return render(request, template, context)

## Vista de Detalle

In [ ]:
def product_model_detail_view(request: HttpRequest, product_id: int) -> HttpResponse:
    instance = get_object_or_404(ProductModel, id=product_id)
    template = "ecommerce/product_detail.html"
    context = {"product": instance}
    
    return render(request, template, context)

## Vista de Creacion

In [ ]:
def product_model_create_view(request: HttpRequest) -> HttpResponse:
    form = ProductModelForm(request.POST or None)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Produto creado con exito.")
        return HttpResponseRedirect("/ecommerce/{product_id}/".format(product_id=instance.id))
    template = "ecommerce/product_create.html"
    context = {"form": form}
    return render(request, template, context)

## Vista de Actualizacion

In [ ]:
def product_model_update_view(request: HttpRequest, product_id: int) -> HttpResponse:
    instance = get_object_or_404(ProductModel, id=product_id)
    form = ProductModelForm(request.POST or None, instance=instance)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Produto actualizado con exito.")
        return HttpResponseRedirect("/ecommerce/{product_id}/".format(product_id=instance.id))
    template = "ecommerce/product_update.html"
    context = {"form": form}
    return render(request, template, context)

## Vista de Eliminar

In [ ]:
def product_model_delete_view(request: HttpRequest, product_id: int) -> HttpResponse:
    instance = get_object_or_404(ProductModel, id=product_id)
    if request.method == "POST":
        instance.delete()
        messages.success(request, "Produto eliminado con exito.")
        return HttpResponseRedirect("/ecommerce/")
    template = "ecommerce/product_delete.html"
    context = {"product": instance}
    
    return render(request, template, context)

## Busqueda en Vista de Listado

- Crea template ecommerce/product_search.html
- Update view.py

In [ ]:
@login_required(login_url='/admin/login/')
def product_model_list_view(request: HttpRequest) -> HttpResponse:
    query = request.GET.get("q", None)
    queryset = ProductModel.objects.all()
    if query is not None:
        queryset = queryset.filter(
            Q(title__icontains=query) |
            Q(description__icontains=query)
        )
    products = queryset
    template = "ecommerce/product_list.html"
    context = {"products": products}
    
    if request.user.is_authenticated:
        template = "ecommerce/product_list.html"
    else:
        template = "ecommerce/product_list_public.html"
    
    return render(request, template, context)